## 1. 引言

前面几个章节我们都是在讲模型各个组件如何构造，每个组件背后的技术原理及其在整个模型中所起的作用，包括分词器、自注意力、LLM架构等。

从本节开始，我们会进入训练部分，开始关注如何对模型进行训练。在训练模型之前，我们需要先构造数据集，那预训练需要什么样的数据呢？

## 2. 预训练需要什么数据？

预训练任务的目的是让GPT模型学会预测下一个词，具体来讲，是通过观察句子中前面的词来学习预测下一个即将出现的词（如下图所示）。
> `预测下一个词`这种方法帮助模型理解词语和短语在语言中通常是如何组合在一起的，从而提升了其对上下文的把握能力，使得生成的文本更加自然流畅与符合语法规则。

![下一个词预测](./img/3-3.jpg)

这是一种自我标记的数据，可以使用句子中下一个词作为模型应该预测的标签，这类标签可以在训练过程中`即时`创建。这意味着我们不需要为训练数据显式的收集标签，可以直接利用大量未标记的文本数据集，来训练大型语言模型。

所以，预训练数据的输入应该是一串文本，预期输出应该是next token。那我们的任务就是把所有输入数据改造成文本串和next token两两映射的输入-目标(input-target)对，如下图所示。

![预训练输入-目标对](./img/3-1.jpg)

给定一个文本，我们需要从中提取输入子文本串，LLM的任务是预测输入子串之后的下一个token。

## 3.基于滑动窗口的数据采样

我们知道，模型的上下文窗口都是有长度限制的，因此我们需要对文本进行采样，以生成符合长度要求的输入-目标对。

下面我们将以存放在`text`变量的文本串为例，详细来绍数据采样的过程（为方便演示，将上下文窗口的长度设置为4）。

In [32]:
context_length = 4
text = "In the heart of the city stood the old library, a relic from a bygone era. Its stone walls bore the marks of time, and ivy clung tightly to its facade ..."

使用之前训练好的分词器对文本进行序列化。

In [35]:
from transformers import AutoTokenizer

tokenizer_path = "/data2/minigpt/models/tokenizer_v3"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
token_ids = tokenizer.encode(text)
print("tokens num:", len(token_ids))
print("tokens:", token_ids) 

tokens num: 44
tokens: [1701, 276, 3471, 303, 276, 2878, 16984, 276, 5294, 12217, 14, 260, 321, 1111, 719, 260, 645, 73, 1045, 14622, 16, 10236, 18392, 13943, 297, 899, 276, 21463, 303, 1100, 14, 281, 223, 430, 91, 940, 6075, 14416, 369, 290, 1113, 2617, 1668, 18952]


In [27]:
tokenizer.decode(token_ids[:30])

'In the heart of the city stood the old library, a relic from a bygone era. Its stone walls bore the marks of time'

创建预训练任务需要的输入-目标对时，我们用两个变量x、y分别表示输入token序列和目标token序列，目标序列是由输入序列中每个token向右移动一个位置构成。

In [36]:
x = token_ids[0: context_length]
y = token_ids[1: context_length+1]
print("x:", x)
print("y:       ", y)

x: [1701, 276, 3471, 303]
y:        [276, 3471, 303, 276]


> 上面的x、y就组成了一条预训练数据，x 是模型接收到的输入序列，y 是预期的模型输出序列，也就是LLM要完成的预测任务。

如果按照context_length的长度继续向右滑动，每次滑动一个单位，我们还可以构建更多的预测任务。

In [29]:
for i in range(context_length+1):
    x = token_ids[i: i + context_length]
    y = token_ids[i+1: i+context_length+1]
    print(f"{x} --> {y}")

[1701, 276, 3471, 303] --> [276, 3471, 303, 276]
[276, 3471, 303, 276] --> [3471, 303, 276, 2878]
[3471, 303, 276, 2878] --> [303, 276, 2878, 16984]
[303, 276, 2878, 16984] --> [276, 2878, 16984, 276]
[276, 2878, 16984, 276] --> [2878, 16984, 276, 5294]


也可以把这些token_id用文本来表示。

In [30]:
for i in range(context_length + 1):
    x = token_ids[i: i + context_length]
    y = token_ids[i+1: i+context_length+1]
    print(f"{tokenizer.decode(x)} --> {tokenizer.decode(y)}")

In the heart of -->  the heart of the
 the heart of the -->  heart of the city
 heart of the city -->  of the city stood
 of the city stood -->  the city stood the
 the city stood the -->  city stood the old


> 注：`-->`左边内容是LLM接收到的输入序列`x`，`-->`右边内容是LLM应该预测的输出序列`y`。

但是这些训练数据之间有很多数据重叠（例如：第一个输入序列`In the heart of`后三个token和第二个输入序列`the heart of the`的前三个token相同），会导致模型进行很多重复的推理和损失计算。为了避免重叠，可以将滑动步长设为`context_length`.

In [31]:
for i in range(0,context_length*5,context_length):
    x = token_ids[i: i + context_length]
    y = token_ids[i+1: i+context_length+1]
    print(f"{tokenizer.decode(x)} --> {tokenizer.decode(y)}")

In the heart of -->  the heart of the
 the city stood the -->  city stood the old
 old library, a -->  library, a re
 relic from a --> lic from a by
 bygone era --> gone era.


这个构建`输入-目标对`的过程就是一个长度为`context_length`的窗口不断向右滑动的过程（如下图所示），因此类似这样的数据采样过程被称为`基于滑动窗口的数据采样`。
![输入-目标对](./img/3-2.jpg)


## 4.数据集加载

#### 4.1 数据集读取
先查看一下我们将要使用的预训练数据集。

In [37]:
!ls -n /data2/minigpt/dataset/pretrain/mobvoi_seq_monkey_general_open_corpus.jsonl

-rw-rw-r-- 1 1010 1010 33391566427 Jan 30  2024 /data2/minigpt/dataset/pretrain/mobvoi_seq_monkey_general_open_corpus.jsonl


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


这个数据集达到33GB，一般的机器内存都无法一次性加载，所以这里在读取数据时引入`yield`生成器模式来迭代返回小批量数据集，通过max_size参数来指定一次迭代的批量数据大小（字符为单位）。

> 注：数据集是一个jsonl格式的文本文件，每一行是一条json格式的数据，文本内容放在text字段中。

In [38]:
import json

def read_text_dataset(data_path, max_size=100*1024*1024):
    with open(data_path, 'r', encoding='utf-8') as f:
        current_size = 0
        current_texts = []
        while True:
            line = f.readline()
            if not line:
                if current_texts:
                    yield current_texts
                break

            data = json.loads(line)
            current_texts.append(data['text'])
            current_size += len(data['text'])
            if current_size >= max_size:
                yield current_texts
                current_texts = []
                current_size = 0

pretrain_data_path = "/data2/minigpt/dataset/pretrain/mobvoi_seq_monkey_general_open_corpus.jsonl"
text_dataset = read_text_dataset(pretrain_data_path)
texts = next(text_dataset)
texts[0], len(texts)

('在查处虚开增值税专用发票案件中，常常涉及进项留抵税额和税款损失的认定和处理。在计算税款损失时，要不要将进项留抵税额包括在内？\n对此，实务中存在意见分歧。\n有人主张归并，即计算税款损失时包括进项留抵税额；\n有人主张剥离，即计算税款损失时剔除进项留抵税额。分析这个问题，需要确定进项留抵税额与税款损失之间是什么关系。\n理清这二者之间的关系，首先需要了解增值税的概念和其抵扣机制。增值税是以商品（货物、服务等）在流转过程中产生的增值额作为计税依据而征收的一种流转税。为避免重复征税，在增值税中存在抵扣链条机制。\n一般而言，交易上游企业缴纳的税额，交易下游企业可以对相应的税额进行抵扣。\n对增值税一般纳税人来说，其购进货物、服务等取得增值税专用发票，发票上的税额是进项税额。\n其出售货物、服务等，向购买方开具增值税专用发票，发票的税额是销项税额。\n一般情况下，销项税额减去进项税额的金额是应纳税额，企业根据应纳税额按期申报纳税。\n其次需要了解进项留抵税额的概念及产生原因。\n在计算销项税额和进项税额的差额时，有时会出现负数，即当期进项税额大于当期销项税额。这个差额在当期未实现抵扣，为进项留抵税额，在以后纳税人有销项税额时再进行抵扣。\n企业产生进项留抵税额的主要原因是其进项税额和销项税额时间上的不一致。\n例如，企业前期集中采购货物和服务，投资大，销项税率低于进项税率等。\n从税款抵扣的角度看，进项留抵税额只是购进的这部分进项税额参与到增值税应纳税额的计算过程中，但是其对应的进项税额抵扣还未真正实现，一般要等到其未来有相应的销项税额时，才能真正实现进项税额抵扣。\n可见，进项留抵税额处于不确定状态，能否抵扣受到很多因素影响，例如企业经营中断，没有销项税额，这时进项留抵税额就无法实现抵扣。但如果企业按照税收政策规定申请进项留抵退税，进项税额抵扣就随之实现。\n最后需要了解税款损失的概念。\n税款损失，通常是指因虚开增值税专用发票，导致国家税款被骗或者流失的金额。关于税款损失，实务中有多种表述。\n例如，北京大学法学院教授陈兴良曾谈到虚开行为本身不会造成国家税款损失，只有利用发票抵扣时才会造成国家税款损失。刘兵等编著的《虚开增值税专用发票案例司法观点和案例解析》一书中提到：“给国家税款造成损失的数额，实际上就是被骗取的国家税款在侦查终结以前无法追回的部分。”\n赵清海

#### 4.2 数据集处理

为了给模型训练提供标准化的输入-输出数据对，我们需要封装一个数据集`PretrainDataset`。主要做以下几件事：
1. 文本序列化，将所有的文本都转换为token ID序列；
2. 滑动窗口采样，按照指定步长stride向右滑动，将token序列切分成一个个max_tokens长度的input序列； 
3. 生成input-target数据对，每个输入序列的生成目标是序列之后的下一个token；
4. 标准化数据访问，实现`__len__`和`__getitem__`方法，兼容pytorch的DataLoader； 

In [48]:
tokenizer.unk_token

'<|endoftext|>'

In [53]:
import torch
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader

class PretrainDataset(Dataset):
    def __init__(self, texts: list, tokenizer, max_tokens, stride=1, limit=-1):
        self.max_tokens = max_tokens
        self.stride = stride
        self.tokenizer: AutoTokenizer = tokenizer
        separator = self.tokenizer.unk_token
        # 文本转换为token ID序列，并用padding_token拼到一起
        token_ids = tokenizer.encode(separator.join(texts)+separator)
        self.input_set = []
        self.target_set = []
        for j in range(0, len(token_ids) - self.max_tokens, self.stride):
            input_ids = token_ids[j: j + self.max_tokens]
            target_ids = token_ids[j+1: j + self.max_tokens + 1]
            self.input_set.append(input_ids)
            self.target_set.append(target_ids)
            if limit > 0 and len(self.input_set) >= limit:
                break
        
    def __len__(self):
        return len(self.input_set)
        
    def __getitem__(self, i):
        return torch.tensor(self.input_set[i]), torch.tensor(self.target_set[i])

> 注：上面这个`PretrainDataset`是基于pytorch的Dataset类来构建，这个基类规定了如何从数据集中抽取单个样本。
> 注：separator是不同文本串之间的分隔符，目的是训练时帮助模型正确区分不同的文本数据，这里使用`<|endoftext|>`作为分隔符。

再基于pytorch提供的Dataloader类来构建数据加载器。

In [54]:
import time

start_time = time.time()
context_length = 1024
ds1 = PretrainDataset(texts, tokenizer, context_length, context_length)
print("len(ds1):", len(ds1), ", use time: ", time.time()-start_time)
dataloader = DataLoader(ds1, batch_size=4, shuffle=True, drop_last=True)
data = next(iter(dataloader))
data

len(ds1): 74406 , use time: %.2f 374.4017357826233


[tensor([[ 4244,  6732,   550,  ...,  8419,  6088, 19271],
         [ 4187,  1883,   626,  ..., 16067,   819,  4825],
         [ 4430,   355,  4430,  ...,    19,   371,   544],
         [  368,  4926,   413,  ...,  2663,  7080,  6115]]),
 tensor([[ 6732,   550,   252,  ...,  6088, 19271, 14891],
         [ 1883,   626,   270,  ...,   819,  4825, 11639],
         [  355,  4430,  2287,  ...,   371,   544,  3078],
         [ 4926,   413,   828,  ...,  7080,  6115,   269]])]

In [19]:
tokenizer.special_tokens_map['unk_token']

'<|endoftext|>'

这样分段加载的数据集虽然解决了大数据量下的内存不足问题，但是在实际应用中还会存在两个问题：
1. 增加了数据访问的复杂性，整个数据集需要先分段加载，再在分段内构造Dataset进行双层循环迭代，并且无法有效预估整个训练的小批量迭代step数；
2. 一份数据通常都会反复训练多轮，每轮训练都需要对文本数据进行分词和序列化，而这个过程又恰恰比较消耗时间和计算资源，导致重复冗余计算； 

## 5. 高效数据加载
针对上面的问题，有一种解决办法是将**所有数据先一次性序列化为二进制格式**，之后使用**numpy的内存映射**功能实现高效的数据加载。这种方式可以避免在内存中加载整个数据集，同时又保证了token序列化这种耗时耗资源的过程只会执行一次。

#### 5.1 数据序列化
下面是数据预处理的函数`texts_to_bin`，接收jsonl数据路径作为输入目录、.bin文件路径作为输出目录，将所有文本经过序列化后拼接到一起，并转换为字节流写入二进制文件。

In [20]:
import numpy as np

def texts_to_bin(input_path, output_path, tokenizer, content_key="content"):
    bos_token = tokenizer.special_tokens_map['bos_token']
    eos_token = tokenizer.special_tokens_map['eos_token']
    max_buffered_length = 1 * 1024 * 1024
    with open(input_path, "r", encoding="utf-8") as reader:
        with open(output_path, "wb") as writer:
            buffered_ids = []
            i = 0
            while True:
                line = reader.readline()
                if not line:
                    break
                content = json.loads(line).get(content_key, "")
                if not content:
                    continue
                
                # 将数据序列化为二进制格式
                tokenized = tokenizer(bos_token + content + eos_token)
                buffered_ids += tokenized["input_ids"]
                if len(buffered_ids) >= max_buffered_length:
                    arr = np.array(buffered_ids, dtype=np.uint16)
                    writer.write(arr.tobytes())
                    buffered_ids.clear()
                    i += 1
                    print(f"write {i}m bytes") if i % 100 == 0 else None
            # 处理最后一段不满max_buffer_length的token序列
            if len(buffered_ids) > 0:
                arr = np.array(buffered_ids, dtype=np.uint16)
                writer.write(arr.tobytes())
                print(f"write arr: {len(arr)}")

In [ ]:
执行过程如下：

In [21]:
%%time 

input_path = "/data2/minigpt/dataset/pretrain/mobvoi_seq_monkey_general_open_corpus.jsonl"
output_path = "/data2/minigpt/dataset/pretrain/mobvoi_seq_monkey_general_open_corpus.bin"
texts_to_bin(input_path, output_path, tokenizer, content_key="text")

write 10000m bytes
write 20000m bytes
write 30000m bytes
write 40000m bytes
write 50000m bytes
write 60000m bytes
write 70000m bytes
write 80000m bytes
write 90000m bytes
write 100000m bytes
write 110000m bytes
write 120000m bytes
write 130000m bytes
write 140000m bytes
write 150000m bytes
write 160000m bytes
write 170000m bytes
write 180000m bytes
write 190000m bytes
write 200000m bytes
write 210000m bytes
write 220000m bytes
write 230000m bytes
write 240000m bytes
write 250000m bytes
write 260000m bytes
write 270000m bytes
write 280000m bytes
write 290000m bytes
write 300000m bytes
write 310000m bytes
write 320000m bytes
write 330000m bytes
write 340000m bytes
write 350000m bytes
write 360000m bytes
write 370000m bytes
write 380000m bytes
write 390000m bytes
write 400000m bytes
write 410000m bytes
write 420000m bytes
write 430000m bytes
write 440000m bytes
write 450000m bytes
write 460000m bytes
write 470000m bytes
write 480000m bytes
write 490000m bytes
write 500000m bytes
write 510

上面代码执行后，33GB的数据已经一次性序列化后二进制文件中，接下来是封装一个数据集类来高效加载数据。

#### 5.2 数据加载
这里会用到一个np.memmap技术，这是一种将文件映射到内存的技术，它允许用户像访问普通内存中的numpy数组一样访问磁盘上的整个数据集，但实际上只有少部分数据从磁盘加载到了内存中，这种技术非常适合处理大型数据集。

In [55]:
class PretrainBinaryDataset(Dataset):
    def __init__(self, data_path, max_tokens):
        with open(data_path) as f:
            f.seek(0, 2)
            self.total_tokens = f.tell() // np.dtype("uint16").itemsize
            print(f"total_tokens: {self.total_tokens}")
        
        self.data = np.memmap(data_path, dtype=np.uint16, shape=(self.total_tokens//max_tokens, max_tokens))

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        assert isinstance(index, int)
        item = self.data[index]
        input = item[:-1].astype(np.int64)
        target = item[1:].astype(np.int64)  # 在计算交叉熵损失时要求目标输出为长整型
        return torch.from_numpy(input), torch.from_numpy(target)

部分代码释义如下：
- `f.seek(0, 2)`:将文件的读取指针移动到文件的末尾（0表示从当前指针移动，2表示从文件末尾移动）; 
- `f.tell()`: 返回当前读取指针的位置（即文件的总字节数），然后我们用字节数除以 np.dtype("uint16").itemsize 获取总tokens数，表示文件中可以读取的token数量; 
- `shape=(self.total_tokens//max_tokens, max_tokens)`: 将数据划分成多个样本，每个样本的大小为 max_tokens(即context_length)，样本的数量为 self.total_tokens // max_tokens。

测试高效数据加载类的加载速度：

In [56]:
%%time

dataset_path = "/data2/minigpt/dataset/pretrain/mobvoi_seq_monkey_general_open_corpus.bin"
ds = PretrainBinaryDataset(dataset_path, context_length)
for i in range(74406):
    item = ds[i]
    pass

total_tokens: 8350205240
CPU times: user 1.1 s, sys: 101 ms, total: 1.2 s
Wall time: 1.23 s


通过对比，同样的数据条数(74406条），第一个数据集类加载耗时374s，第二个高效数据类只用了1.23s，加载速度提高了将近300倍。

**小结**：本文从预训练需要什么样的数据开始讨论，详细介绍了基于滑动窗口的数据采样技术。之后对比了两种数据加载器，显然基于预序列化加np.memmap技术的加载器在处理大规模数据时具有更显著的优势。这种技术不仅能够有效地减少内存占用，还能以更快的速度进行数据访问，从而加速模型的训练过程。

## 相关资料
- [预训练数据集](http://share.mobvoi.com:5000/sharing/O91blwPkY)